## An Analysis of the 2024 Syracuse Mets

In [ ]:
import pandas as pd
from utils import *

pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

columnsToConvert = ['OBP', 'SLG', 'Whiff%', 'BA']
leaguewideAAAStatsDf = convertColumnsToNumeric(leaguewideAAAStatsDf, columnsToConvert)

leaguewideAAAStatsDf['OPS'] = leaguewideAAAStatsDf['OBP'] + leaguewideAAAStatsDf['SLG']

completeMetsAAAStats = joinSavantBbref(metsAAAbballSavant, metsAAAbballRef)
completeMetsAAAStats['RC'] = ((completeMetsAAAStats['Hits'] + completeMetsAAAStats['BB']) * completeMetsAAAStats['TB'])/(completeMetsAAAStats['AB'] + completeMetsAAAStats['BB'])

completeMetsAAAStats = completeMetsAAAStats[(completeMetsAAAStats['PA'] > 90) & (completeMetsAAAStats['Age'] <= 26)]

leaguewideAAAStatsDf = calculateSavantStats(leaguewideAAAStatsDf)
leaguewideAAAStatsDf = leaguewideAAAStatsDf[leaguewideAAAStatsDf['PA'] > 90]

print(completeMetsAAAStats[defaultInfo + slashLine + ['HR', 'RC']].sort_values(by='OPS', ascending=False))
summary_df = pd.concat([
    leaguewideAAAStatsDf['BA'].describe(),
    leaguewideAAAStatsDf['OBP'].describe(),
    leaguewideAAAStatsDf['SLG'].describe(),
    leaguewideAAAStatsDf['OPS'].describe()
], axis=1)

# Rename the columns to reflect the stats
#summary_df.columns = ['BA', 'OBP', 'SLG', 'OPS']

print(summary_df)

#completeMetsAAAStats.plot(kind='bar', x='Player', y='HR', title='Home Runs by Player', legend=False)

In [ ]:
print(completeMetsAAAStats[defaultInfo + slashLine + actualVsEx].sort_values(by='OPS', ascending=False))

In [ ]:
print(completeMetsAAAStats[defaultInfo + slashLine + plateDiscipline].sort_values(by='OPS', ascending=False))

In [ ]:
print(print(completeMetsAAAStats[defaultInfo + slashLine + powerQuality].sort_values(by='OPS', ascending=False)))